In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON

def parse_group_concat(value):
    if value is None or value == "":
        return []
    return value.split("|")

def extract_year(date_string):
    """
    Extrait l'année d'une date Wikidata du type '+1887-01-28T00:00:00Z'
    et la retourne sous forme d'entier.
    """
    if not date_string:
        return None
    
    year_str = date_string.strip("+").split("-")[0]
    
    # Convertit en int pour permettre le tri
    try:
        return int(year_str)
    except ValueError:
        return None


def get_first_and_last_year(dates):
    """
    Prend une liste de dates Wikidata et renvoie :
    - première année (min)
    - dernière année (max)
    """
    if not dates:
        return None, None

    # Conversion des dates Wikidata en années entières
    years = [extract_year(d) for d in dates if d]

    # Filtre les valeurs invalides
    years = [y for y in years if y is not None]

    if not years:
        return None, None

    # 🔥 TRI CHRONOLOGIQUE
    years = sorted(years)

    # Retourne la première et la dernière année
    return years[0], years[-1]


def get_first_or_none(values):
    if not values:
        return None
    return values[0]

def get_monument_data(qid):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    query = f"""
    SELECT
        (GROUP_CONCAT(DISTINCT STR(?inception); SEPARATOR="|") AS ?inception)
        (GROUP_CONCAT(DISTINCT STR(?instanceOf); SEPARATOR="|") AS ?instanceOf)
        (GROUP_CONCAT(DISTINCT STR(?image); SEPARATOR="|") AS ?image)
        (GROUP_CONCAT(DISTINCT STR(?country); SEPARATOR="|") AS ?country)
        (GROUP_CONCAT(DISTINCT STR(?city); SEPARATOR="|") AS ?city)
        (GROUP_CONCAT(DISTINCT STR(?lat); SEPARATOR="|") AS ?lat)
        (GROUP_CONCAT(DISTINCT STR(?lon); SEPARATOR="|") AS ?lon)
        (GROUP_CONCAT(DISTINCT STR(?architect); SEPARATOR="|") AS ?architect)
    WHERE {{
      VALUES ?item {{ wd:{qid} }}

      OPTIONAL {{ ?item wdt:P571 ?inception. }}
      OPTIONAL {{ ?item wdt:P31 ?instanceOf. }}
      OPTIONAL {{ ?item wdt:P18 ?image. }}
      OPTIONAL {{ ?item wdt:P17 ?country. }}
      OPTIONAL {{ ?item wdt:P131 ?city. }}

      OPTIONAL {{
        ?item wdt:P625 ?coords.
        BIND(geof:latitude(?coords) AS ?lat)
        BIND(geof:longitude(?coords) AS ?lon)
      }}

      OPTIONAL {{ ?item wdt:P84 ?architect. }}
    }}
    GROUP BY ?item
    """

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    row = results["results"]["bindings"][0]

    # Lecture des variables brutes
    inception_list   = parse_group_concat(row.get("inception", {}).get("value"))
    instance_of_list = parse_group_concat(row.get("instanceOf", {}).get("value"))
    image_list       = parse_group_concat(row.get("image", {}).get("value"))
    country_list     = parse_group_concat(row.get("country", {}).get("value"))
    city_list        = parse_group_concat(row.get("city", {}).get("value"))
    lat_list         = parse_group_concat(row.get("lat", {}).get("value"))
    lon_list         = parse_group_concat(row.get("lon", {}).get("value"))
    architect_list   = parse_group_concat(row.get("architect", {}).get("value"))

    # 🎯 Transformation :
    first_image   = get_first_or_none(image_list)
    first_lat     = float(get_first_or_none(lat_list)) if lat_list else None
    first_lon     = float(get_first_or_none(lon_list)) if lon_list else None
    first_city    = get_first_or_none(city_list)
    first_country = get_first_or_none(country_list)
    first_year, last_year = get_first_and_last_year(inception_list)


    # Variables finales
    return (
        first_year,       # ✔ année de début
        last_year,        # ✔ année de fin
        instance_of_list,
        first_image,      # ✔️ PAS UNE LISTE
        first_country,
        first_city,
        first_lat,        # ✔️ float
        first_lon,        # ✔️ float
        architect_list
    )


# Exemple : Tour Eiffel
(
    first_year,
    last_year,
    instance_of,
    image,
    countries,
    cities,
    lat,
    lon,
    architects
) = get_monument_data("Q243")

print("Année inception (début) :", first_year)
print("Année inception (fin) :", last_year)
print("Instance of :", instance_of)
print("Image :", image)
print("Latitude :", lat)
print("Longitude :", lon)
print("Countries :", countries)
print("Cities :", cities)
print("Architects :", architects)


Année inception (début) : 1887
Année inception (fin) : 1889
Instance of : ['http://www.wikidata.org/entity/Q2319498', 'http://www.wikidata.org/entity/Q1440476', 'http://www.wikidata.org/entity/Q1440300', 'http://www.wikidata.org/entity/Q570116']
Image : http://commons.wikimedia.org/wiki/Special:FilePath/Tour%20Eiffel%20Wikimedia%20Commons.jpg
Latitude : 48.858296
Longitude : 2.294479
Countries : http://www.wikidata.org/entity/Q142
Cities : http://www.wikidata.org/entity/Q259463
Architects : ['http://www.wikidata.org/entity/Q778243']
